In [1]:
from astropy.coordinates import EarthLocation, AltAz, ITRS
from scipy import interpolate
import astropy.units as u
import numpy as np
from astropy.time import Time

In [2]:
from lofarantpos.db import LofarAntennaDatabase

In [3]:
from pyproj import Transformer

In [4]:
db = LofarAntennaDatabase()

In [5]:
loc_rs305 = EarthLocation(*(db.phase_centres["RS305LBA"]*u.m))
loc_de605 = EarthLocation(*(db.phase_centres["DE605LBA"]*u.m))

In [6]:
dummytime = Time.now()

In [7]:
wgs84_to_rds = Transformer.from_crs(4326, 28992)
rds_to_wgs84 = Transformer.from_crs(28992, 4326)

In [8]:
def distance_from_height(obs_altaz, height=100*u.km):
    """From an AltAz pointing, find the distance to where it crosses a plane 100 km above Earth"""
    try_distances = np.linspace(50, 1000) * u.km
    try_altaz = AltAz(location=obs_altaz.location, obstime=dummytime, az=obs_altaz.az, alt=obs_altaz.alt, distance=try_distances)
    try_heights = EarthLocation(*(try_altaz.transform_to(ITRS(obstime=dummytime)).cartesian.xyz)).height
    height_to_distance = interpolate.interp1d(try_heights.to(u.km), try_distances.to(u.km))
    return height_to_distance(height) * u.km

In [9]:
def earthloc_at_height(obs_altaz, height=100*u.km):
    """From an AltAz pointing, find the EarthLocation at which point the line of sight is exactly at a given height (100km)"""
    distance = distance_from_height(obs_altaz, height)
    intersection_altaz = AltAz(location=obs_altaz.location, obstime=dummytime, az=obs_altaz.az, alt=obs_altaz.alt, distance=distance)
    return EarthLocation(*(intersection_altaz.transform_to(ITRS(obstime=dummytime)).cartesian.xyz))

In [10]:
def draw_fov(
    obsloc: EarthLocation, alt0, az0, fwhm=13 * u.deg, height=100 * u.km, num_segments=31
):
    """
    Make well-known-text (WKT) representation of beam at a certain height
    Coordinate system is rijksdriehoeksstelsel valid in NL
    """
    nodes = []
    for angle_offset in np.linspace(0, 360, num_segments) * u.deg:
        alt = alt0 + 0.5 * fwhm * np.cos(angle_offset)
        az = az0 + 0.5 * fwhm * np.sin(angle_offset)
        obs_altaz = AltAz(location=obsloc, obstime=dummytime, alt=alt, az=az)
        loc = earthloc_at_height(obs_altaz, height=height)
        node = wgs84_to_rds.transform(loc.lat.degree, loc.lon.degree, loc.height.value)[:2]
        nodes += [node]
    nodes += [nodes[0]]
    return "POLYGON((" +  ", ".join([f"{node[0]:.2f} {node[1]:.2f}" for node in nodes]) + "))"

In [11]:
for az0 in np.array([195, 210, 225]) * u.deg:
    print(draw_fov(loc_rs305, 30 * u.deg, az0, fwhm=5.16*u.deg, height=100*u.km))

POLYGON((211981.00 399966.27, 210529.72 400005.93, 208974.63 399439.72, 207365.25 398278.90, 205756.39 396552.65, 204207.99 394308.91, 202782.37 391609.40, 201545.28 388541.55, 200562.04 385216.54, 199893.42 381769.09, 199590.32 378354.55, 199688.39 375144.61, 200202.48 372317.71, 201117.77 370029.92, 202393.91 368420.80, 203964.09 367591.22, 205740.00 367592.36, 207619.62 368420.16, 209496.80 370016.58, 211270.70 372277.03, 212855.41 375070.38, 214180.74 378231.85, 215198.12 381592.99, 215880.06 384992.62, 216217.46 388283.24, 216216.21 391336.02, 215893.68 394042.97, 215276.18 396320.41, 214394.37 398097.97, 213283.12 399324.28, 211981.00 399966.27, 211981.00 399966.27))
POLYGON((175238.60 414367.42, 173847.04 414781.22, 172198.41 414636.66, 170343.46 413931.79, 168342.69 412680.62, 166266.41 410913.92, 164190.79 408675.21, 162201.95 406031.93, 160391.78 403074.56, 158853.83 399917.52, 157677.47 396697.71, 156941.57 393571.73, 156706.63 390708.10, 156998.73 388261.44, 157815.01 38637

In [12]:
de605_pointings = []
for rs305_az in np.array([195, 210, 225]) * u.deg:
    rs305_altaz_far = AltAz(location=loc_rs305, alt=30 * u.deg, az=rs305_az)
    rs305_altaz_near = AltAz(
        location=loc_rs305,
        alt=30 * u.deg,
        az=rs305_az,
        distance=distance_from_height(rs305_altaz_far, 100 * u.km),
        obstime=dummytime,
    )
    de605_altaz_near = rs305_altaz_near.transform_to(
        AltAz(location=loc_de605, obstime=dummytime)
    )
    de605_pointings.append(de605_altaz_near)

In [13]:
de605_pointings

[<AltAz Coordinate (obstime=2020-12-07 21:12:32.636132, location=(4005681.742, 450968.282, 4926457.67) m, pressure=0.0 hPa, temperature=0.0 deg_C, relative_humidity=0.0, obswl=1.0 micron): (az, alt, distance) in (deg, deg, m)
     (343.06082359, 56.67672373, 119108.1677347)>,
 <AltAz Coordinate (obstime=2020-12-07 21:12:32.636132, location=(4005681.742, 450968.282, 4926457.67) m, pressure=0.0 hPa, temperature=0.0 deg_C, relative_humidity=0.0, obswl=1.0 micron): (az, alt, distance) in (deg, deg, m)
     (322.90088284, 44.90473886, 140371.98250258)>,
 <AltAz Coordinate (obstime=2020-12-07 21:12:32.636132, location=(4005681.742, 450968.282, 4926457.67) m, pressure=0.0 hPa, temperature=0.0 deg_C, relative_humidity=0.0, obswl=1.0 micron): (az, alt, distance) in (deg, deg, m)
     (318.08824405, 34.6037656, 173076.6850057)>]

In [14]:
for de605_pointing in de605_pointings:
    print(
        draw_fov(
            loc_de605,
            de605_pointing.alt,
            de605_pointing.az,
            fwhm=6.46 * u.deg,
            height=100 * u.km,
        )
    )

POLYGON((210574.52 377705.07, 211162.90 378058.00, 211639.78 378698.97, 211991.53 379607.37, 212206.13 380752.40, 212273.10 382093.12, 212183.96 383578.92, 211933.36 385150.42, 211520.81 386740.99, 210952.59 388279.02, 210243.55 389691.11, 209418.42 390906.09, 208511.98 391859.65, 207567.97 392499.23, 206636.53 392788.55, 205770.37 392710.95, 205020.18 392271.12, 204429.95 391494.68, 204032.91 390425.80, 203848.80 389123.14, 203882.72 387654.88, 204125.62 386093.56, 204556.30 384511.50, 205144.22 382977.08, 205852.91 381552.23, 206643.26 380290.90, 207476.43 379238.33, 208316.14 378430.81, 209130.07 377895.56, 209890.56 377650.70, 210574.52 377705.07, 210574.52 377705.07))
POLYGON((174095.41 392670.17, 174779.01 393466.83, 175185.81 394569.29, 175301.90 395944.40, 175119.79 397548.46, 174638.80 399326.82, 173866.18 401213.84, 172818.76 403133.73, 171524.92 405002.42, 170026.38 406730.70, 168379.32 408228.87, 166654.00 409412.77, 164932.62 410210.45, 163305.39 410568.49, 161861.51 41046